In [ ]:
#! If you want to force use of CPU (before import of tensorflow)
import os
# os.environ['CUDA_VISIBLE_DEVICES']='-1'

#! import as tf
import tensorflow as tf

#! If you want to limit number of threads, the mechine has 32 virtual cores (16 physical ones)
print('thread inter = ', tf.config.threading.get_inter_op_parallelism_threads() )
print('thread intra = ', tf.config.threading.get_intra_op_parallelism_threads() )
 
tf.config.threading.set_inter_op_parallelism_threads(4)
tf.config.threading.set_intra_op_parallelism_threads(4)

#! If you want to limit the GPU memory usage, the machine has ~24GB. For most applications a few GBs is enough.
#! If you max out what you specify progressively up the threshhold.
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.set_logical_device_configuration(
    physical_devices[0],
    [tf.config.LogicalDeviceConfiguration(memory_limit=6000)])

  logical_devices = tf.config.list_logical_devices('GPU')

except:
  # Invalid device or cannot modify logical devices once initialized.
  pass

#! Will print out what devices you are using
tf.config.list_physical_devices()



import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm

from mlkernels.tensorflow import Matern32, EQ, Matern12, Matern52


from LFM.NeuralNet import NeuralN
from LFM.utilities import reparameterize, log_normal_pdf, expHighLowFunc_01
from LFM.GICNet    import FFLayer, KIPLayer, GICNet

pi  = tf.constant(np.pi, tf.float32)
tfd = tfp.distributions
tfb = tfp.bijectors


dirname='data-phys_data-lfmDiag-PoissonNonLinear1D/'
os.makedirs(dirname, exist_ok=True)



In [ ]:

dimU = 1
dimZ = 5
dimW = 1
dimX = 1

            # x      
domain     = tf.constant( [-1., 1.] )

nxSamples  = 30

zDist = tfd.Uniform(low=tf.constant(-1., shape=[dimZ]), high=tf.constant(1., shape=[dimZ]))
wDist = tfd.Uniform(low=tf.constant( 1., shape=[dimW]), high=tf.constant(2., shape=[dimW]))
e_bound = 1e-5
xDist = tfd.Uniform(low=domain[0]+e_bound, high=domain[1]-e_bound) 

b     = zDist.sample()
xvect = xDist.sample(sample_shape=(nxSamples, dimX))

def raveled1DDomainGrid(resolution):
    x  = tf.linspace(domain[0], domain[1], resolution[0])
    return x[:, None]


In [ ]:
# dirnameGT = 'data-trueEq--PoissonNonLinear1D_1000'
# dirnameGT = '../../fenics/exp/poisson1DNonLinear-Comparisons/data-groundTruth--PoissonNonLinear1D_1000'
dirnameData = '../../../fenics/poisson1DNonLinear/data-groundTruth_2--PoissonNonLinear1D_1000'


dataU = tf.constant( np.loadtxt(dirnameData+'/' + 'allGroundTruthU.dat' ), dtype=tf.float32)
dataZ = tf.constant( np.loadtxt(dirnameData+'/' + 'allGroundTruthZ.dat' ), dtype=tf.float32)
dataW = tf.constant( np.loadtxt(dirnameData+'/' + 'allGroundTruthW.dat' ), dtype=tf.float32)
dataX = tf.constant( np.loadtxt(dirnameData+'/' + 'FenicsMeshX.dat'     ), dtype=tf.float32)

sigma_y         = tf.constant(0.05)
nData_input     = 1000
# dataY           = ( dataU + tf.random.normal(stddev=sigma_y, shape=dataU.shape) )[:, 1:-1]
dataY           = ( dataU + tf.random.normal(stddev=sigma_y, shape=dataU.shape) )[:nData_input, 20:-20:3]

batch_size_data = 50

# data_single_X     = tf.reshape(dataX, [-1,1])[1:-1,:]
data_single_X     = tf.reshape(dataX, [-1,1])[20:-20:3, :]

print('data_single_X.shape', data_single_X.shape)

dimY            = dataY.shape[1]
nData_y         = dataY.shape[0]
print('dimY', dimY)

dataset_data_YZWX_stack = tf.concat([dataY, dataZ[:nData_input,:], dataW[:nData_input,None]], 1)
print('dataset_data_YZWX_stack.shape', dataset_data_YZWX_stack.shape)

dataset_data_YZWX       = tf.data.Dataset.from_tensor_slices( dataset_data_YZWX_stack )
dataset_data_YZWX_batched = dataset_data_YZWX.batch(batch_size=batch_size_data)

for i in range(5):
    plt.scatter(data_single_X, dataY[i], s=5)
    plt.plot(dataX, dataU[i])
plt.tight_layout()
plt.xlabel('x')
plt.ylabel('y')
plt.savefig('5Data_ySamples_trueLine_scatterData.pdf')
plt.show()

In [ ]:
# Define alpha network

dim_DeepO_out  = dimU * 2

alphaNet  = NeuralN('alpha', np.array([ 1, 1, dimZ]) )

numFreq = 100

nhAlpha = 300

alphaNet.NN = tf.keras.Sequential([ tf.keras.layers.Dense( nhAlpha, input_dim=( numFreq * 2 + dimZ + dimW ), activation='swish'),
                                    tf.keras.layers.Dense( nhAlpha, activation='swish'),
                                    tf.keras.layers.Dense( nhAlpha, activation='swish'),
                                    tf.keras.layers.Dense( nhAlpha, activation='swish'),
                                    tf.keras.layers.Dense( nhAlpha, activation='swish'),
                                    tf.keras.layers.Dense( nhAlpha, activation='swish'),
                                    tf.keras.layers.Dense( nhAlpha, activation='swish'),
                                    tf.keras.layers.Dense( dim_DeepO_out ) ])

freqsX = tf.linspace(0., 1., numFreq)
freqs  = expHighLowFunc_01(2., 0.001, freqsX)
tf.print('all freqs FFN \n', freqs, summarize=-1) 
fLayer = [ FFLayer( freqs ) ]

alphaNet.lower_alpha = tf.constant(1e-5)
alphaNet.upper_w     = tf.constant(1.)

def alphaMap(self, z, w, x):

        z_tile  = tf.tile(tf.reshape(z, [1,-1]), [x.shape[0], 1])
        w_tile  = tf.tile(tf.reshape(w, [1,-1]), [x.shape[0], 1])
        xFreq   = fLayer[0]( x )
        x_input = tf.concat([xFreq, z_tile, w_tile], 1)
        mapped  = self.NN( x_input )
        
        mean, logvar = mapped[:,:dimU], mapped[:,dimU:2*dimU]
        
        tf.debugging.assert_all_finite(mean, 'mean not finite')
        tf.debugging.assert_all_finite(logvar, 'logvar not finite')

        logvar = self.dBoundwalpha_log( logvar )

        print('alpha mean.shape = ',   mean.shape)
        print('alpha logvar.shape = ', logvar.shape)

        return mean, logvar

from types import MethodType
alphaNet.call = MethodType(alphaMap, alphaNet)
alphaNet(tf.constant(1., shape=[dimZ]), tf.constant(1., shape=[dimW]), tf.constant(1., shape=[nxSamples,dimX]))
alphaNet.compile()
alphaNet.NN.summary()
alphaNet.summary()

# alphaNet.save_weights(dirname + '/alphaModel/alphaModel-lfmDiag')
# alphaNet.load_weights(dirname + '/alphaModel/alphaModel-lfmDiag')

# del alphaNet


In [ ]:
# Define beta network

dim_trunc_in_beta   = dimW
dim_trunc_out_beta  = 100 

dim_branch_in_y_beta = 1
dim_branch_P_beta    = 20
dv_beta              = 20 # number of channels -- filters
dim_branch_out_beta  = 100 #dimZ #20 #works


dim_DeepO_in_beta   = dim_trunc_out_beta + dim_branch_out_beta
dim_DeepO_out_beta  = dimZ * 2

kip_res       = tf.constant([20])
grid_kip_beta = raveled1DDomainGrid( kip_res )

gicNetBeta = GICNet( grid_kip_beta, kip_res, dim_branch_in_y_beta, dim_branch_P_beta, dimX  )

gicNetBeta.P_kLayers   = [ tf.keras.layers.Dense( dim_branch_P_beta, input_dim=( dim_branch_in_y_beta ), activation ='swish'),
                           tf.keras.layers.Dense( dim_branch_P_beta, activation ='swish'),
                           tf.keras.layers.Dense( dim_branch_P_beta, activation ='swish'),
                           tf.keras.layers.Dense( dim_branch_P_beta, activation ='swish') ]
                            
gicNetBeta.kipLayers   = []
ellXs = tf.linspace(0., 1., dim_branch_P_beta)
low  = 1e-3
high = 1.
ellVals     = expHighLowFunc_01(low, high, ellXs)
for i in range(dim_branch_P_beta):

        gicNetBeta.kipLayers.append(KIPLayer(EQ(), ellVals[i]))
        print('kip layer {} ell = '.format(i), gicNetBeta.kipLayers[i].ell)


gicNetBeta.FFMlayer    = [ ]

gicNetBeta.preTrunclayers  = [ 
                         tf.keras.layers.Dense( dim_trunc_out_beta, input_dim=( dim_trunc_in_beta ), activation ='swish'), 
                         tf.keras.layers.Dense( dim_trunc_out_beta,                                  activation ='swish'), 
                         tf.keras.layers.Dense( dim_trunc_out_beta,                                  activation ='swish')
                         ]

gicNetBeta.convLayers = [ 
                        tf.keras.layers.Conv1D( dv_beta, (3), strides=(1), activation='swish' ),
                        tf.keras.layers.Conv1D( dv_beta, (3), strides=(1), activation='swish' ),
                        tf.keras.layers.Conv1D( dv_beta, (3), strides=(1), activation='swish' ),
                        tf.keras.layers.Conv1D( dv_beta, (3), strides=(1), activation='swish' ),
                        tf.keras.layers.Conv1D( dv_beta, (3), strides=(1), activation='swish' ),
                        tf.keras.layers.Conv1D( dv_beta, (3), strides=(1), activation='swish' ),
                        tf.keras.layers.Flatten(  ),
                        tf.keras.layers.Dense(500),
                        tf.keras.layers.Dense(dim_branch_out_beta),
                        ]
                        
# nn  = tf.keras.Sequential( gicNetBeta.convLayers  )
# val = tf.random.uniform(shape=[1,kip_res[0], dim_branch_P_beta + dimX])
# nn(val)
# nn.summary()
# nn = None

nhBeta       = 300
deep0_beta   = tf.keras.layers.Dense( nhBeta, input_dim=( dim_DeepO_in_beta ), activation=tf.keras.activations.swish)
deep1_beta   = tf.keras.layers.Dense( nhBeta, activation=tf.keras.activations.swish)
deep2_beta   = tf.keras.layers.Dense( nhBeta, activation=tf.keras.activations.swish)
deep3_beta   = tf.keras.layers.Dense( nhBeta, activation=tf.keras.activations.swish)
deep2_beta   = tf.keras.layers.Dense( nhBeta, activation=tf.keras.activations.swish)
deep3_beta   = tf.keras.layers.Dense( nhBeta, activation=tf.keras.activations.swish)
deepOut_beta = tf.keras.layers.Dense( dim_DeepO_out_beta )
gicNetBeta.DeepOlayers = [ deep0_beta, deep1_beta, deep2_beta, deep3_beta, deepOut_beta ]


betaNet    = NeuralN('beta', np.array([ 1, 1, dimZ]) )
betaNet.NN = gicNetBeta

betaNet.lower_alpha  = tf.constant(1e-5) 
betaNet.upper_w      = tf.constant(1.)

def betaMap(self, u, w, x):

        trunc_in  = tf.reshape(w, [-1,dimW])
        branch_in = tf.concat([x, u], 1)

        mapped = self.NN( branch_in, trunc_in )
        print('mapped.shape = ', mapped.shape)

        mean, logvar = mapped[:,:dimZ], mapped[:,dimZ:2*dimZ]

        tf.debugging.assert_all_finite(mean, 'mean not finite')
        tf.debugging.assert_all_finite(logvar, 'logvar not finite')

        logvar = self.dBoundwalpha_log( logvar )
        print('beta mean.shape = ', mean.shape)
        print('beta logvar.shape = ', logvar.shape)

        return mean, logvar

from types import MethodType
betaNet.call = MethodType(betaMap, betaNet)

mean_alpha, logvar_alpha = alphaNet(tf.constant(1., shape=[dimZ]), tf.constant(1., shape=[dimW]), xvect)
print(mean_alpha.shape)

mean_beta, logvar_beta = betaNet( mean_alpha, tf.constant(1., shape=[dimW]), xvect )
betaNet.compile( )
betaNet.summary( )
betaNet.NN.summary( )

# betaNet.save_weights(dirname + '/betaModel/betaModel-lfmDiag')
# betaNet.load_weights(dirname + '/betaModel/betaModel-lfmDiag')

# del betaNet


In [ ]:

def D(x):
    return tf.cos(0.5 * pi * x)

epsilon_r = tf.Variable(1e-2, trainable=False)

Nz = tf.range(0, dimZ, dtype=tf.float32)

def cheby(n, x):
    return tf.cos( n * tf.math.acos(x) )

def bound( xd ): return cheby(Nz, xd)

def kappa(z, u, x):
    Txn  =  tf.vectorized_map(bound, x)
    return tf.math.log( 1. + tf.exp( tf.reshape(u, [-1,1]) * tf.reshape(Txn @ tf.reshape(z, [-1,1]), [-1,1]) ) ) + 0.1

x = tf.linspace(-1., 1., 100)[:, None]
z = zDist.sample()
plt.plot(tf.linspace(-1., 1., 100), kappa(z, x**0., x) )
plt.show()
plt.close()

def forcing(w, x):
    return w + x * 0.

# @tf.function
def residualFunction( z, w, x ):
    
    # x, t = xrand[:,:1], xrand[:, 1:]
    x = tf.reshape(x, [-1,1])
    z = tf.reshape(z, [-1,1])

    with tf.GradientTape(watch_accessed_variables=False, persistent=True) as t1:
        t1.watch( x )

        with tf.GradientTape(watch_accessed_variables=False, persistent=True) as t2:
            t2.watch( x )

            mean_alpha, logvar_alpha = alphaNet( z, w, x )
            u_untrans = tf.reshape(reparameterize(mean_alpha, logvar_alpha), [-1,1])

            u = D(x) * u_untrans

            k = kappa(z, u, x)
            
        u_x = tf.reshape(t2.gradient(u, x), [-1,1])
        k_x = tf.reshape(t2.gradient(k, x), [-1,1])
    
    u_xx = tf.reshape(t1.gradient(u_x, x), [-1,1])

    f = forcing(w, x)
    
    # true residual
    res = k_x * u_x + k * u_xx + f
    
    print("residualBody.shape", res.shape)
    print('u.shape', u.shape)
    print('mean_alpha.shape', mean_alpha.shape)
    print('logvar_alpha.shape', logvar_alpha.shape )
        
    logP_alpha_u_Z = log_normal_pdf(u_untrans, mean_alpha, logvar_alpha, raxis=0)

    return u_untrans, u, res, logP_alpha_u_Z


def ELBO_phys( zwx ):

    # z     = tf.reshape( zDist.sample( ), [dimZ])
    # w     = tf.reshape( wDist.sample( ), [dimW])
    # # fixed grid
    # # xrand = XTr

    # # random uniform
    # xrand = xDist.sample(sample_shape=[nxSamples, dimX])
    # print("xrand.shape", xrand.shape)

    z,w,xrand = zwx[:dimZ], zwx[dimZ:dimZ+dimW], zwx[dimZ+dimW:]
    xrand     = tf.reshape(xrand, [-1,1])
    print("ELBO xrand.shape", xrand.shape)
    print('ELBO z.shape', z.shape)
    print('ELBO w.shape', w.shape)

    u_untrans, us, residual, logP_alpha_u_Z = residualFunction( z, w, xrand )

    logVarEpsVect  = tf.repeat(2.*tf.math.log(epsilon_r), residual.shape[0] )[:, None]

    logP_r   = log_normal_pdf( residual, tf.constant(0., shape=residual.shape), logVarEpsVect, raxis=0 )

    mean_beta, logvar_beta = betaNet( tf.reshape(u_untrans, [-1,1]), w, xrand )

    print('mean_beta.shape', mean_beta.shape)
    print('mean_beta.shape', logvar_beta.shape)

    logP_beta_Z_u          = log_normal_pdf( tf.reshape(z, [1,-1]), mean_beta, logvar_beta, raxis=1 ) #this is correct dimension

    print('logP_alpha_u_Z.shape', logP_alpha_u_Z.shape)
    print('logP_beta_Z_u.shape', logP_beta_Z_u.shape)
    print('logP_r.shape', logP_r.shape)

    elbo = tf.reduce_mean( logP_r + logP_beta_Z_u -  logP_alpha_u_Z )

    return elbo

zs =  zDist.sample( )
ws =  wDist.sample( )
xs =  xDist.sample( sample_shape=(nxSamples) )

zwx = tf.concat([zs, ws, xs], 0)

ELBO_phys( zwx )

In [ ]:
def ELBO_data( YZW ):

    y, z, w = YZW[:dimY], YZW[dimY:dimY+dimZ], YZW[dimY+dimZ:]
    y       = tf.reshape(y, [-1,1])

    print('y.shape', y.shape)
    print('z.shape', z.shape)
    print('w.shape', w.shape)


    mean_alpha, logvar_alpha = alphaNet( z, w, data_single_X )

    Dvect = D( data_single_X )

    mean_alpha_trans = Dvect * mean_alpha

    var_trans_y  = Dvect ** 2. * tf.exp( logvar_alpha ) + sigma_y**2.

    log_var_trans_y = tf.math.log( var_trans_y )

    logP_Y_zwx       = log_normal_pdf(y, mean_alpha_trans, log_var_trans_y, raxis=0)

    return tf.reshape( logP_Y_zwx, [ ] )

ELBO_data(dataset_data_YZWX_stack[0,:])

In [ ]:
trainableVars   = [ alphaNet.trainable_variables, betaNet.trainable_variables]

@tf.function
def train_step(optimizer, batch_data):

    zs =  zDist.sample( batchSize )
    ws =  wDist.sample( batchSize )
    xs =  xDist.sample( sample_shape=(batchSize, nxSamples) )

    print('zs.shape', zs.shape)
    print('ws.shape', ws.shape)
    print('xs.shape', xs.shape)

    zwxs = tf.concat([zs, ws, xs], 1)

    print('zwxs.shape', zwxs.shape)

    with tf.GradientTape(persistent=True) as tape:
        # posloss, logP_r, logP_beta_Z_u, logP_alpha_u_Z = ELBO( )
        # loss    = - posloss

        losses_phys =   tf.vectorized_map(ELBO_phys, zwxs)
        loss_phys   = - tf.reduce_mean(losses_phys)

        losses_data = tf.vectorized_map( ELBO_data, batch_data )
        loss_data   = - nData_y / batch_size_data * tf.reduce_sum(losses_data)

        print('loss_data = ', loss_data)

        loss = loss_phys + loss_data

    for vars in trainableVars:
        gradients = tape.gradient(loss, vars)
        optimizer.apply_gradients(zip(gradients, vars))

    return loss, loss_data, loss_phys

batchSize = 50

num_iterations   = tf.constant(1_000_000)
epochs           = num_iterations/int(nData_y/batch_size_data)/batchSize

epsilon_r.assign( tf.constant(1e-2) )

intervalELBOsave = 10

lr = tf.keras.optimizers.schedules.ExponentialDecay(
    1e-3, int(num_iterations/batchSize/5), 0.5, staircase=True, name=None
)
optimizer = tf.keras.optimizers.Adam( lr )

elboAll     = []
startTrainingPDDLVM = time.time()
iteration   = tf.constant(0)
for epoch in tf.range(0, epochs ):

    start_time = time.time()
    for batch_data in dataset_data_YZWX_batched:
        loss, loss_data, loss_phys = train_step(optimizer, batch_data)
        iteration += batchSize

    end_time = time.time()

    if epoch % intervalELBOsave == 0:
        elboAll.append( -loss ) 
        print('Iteration: {}, Epoch: {}, ELBO: {:.2f}, loss_phys: {:.2f}, loss_data: {:.2f}, time elapse for current epoch: {:.2f}'
           .format(iteration, epoch, -loss, -loss_phys, -loss_data, end_time - start_time))

endTrainingPDDLVM = time.time()
print('totalTimeTrainingPDLVM = {:.2f}m'.format((endTrainingPDDLVM - startTrainingPDDLVM)/60.))



In [ ]:
ebloAll = tf.stack(elboAll)
LOSS    = np.array(ebloAll)
plt.plot(range(LOSS.shape[0]), LOSS )
plt.show()
print(np.isnan(LOSS).mean())
plt.plot(range(LOSS.shape[0]), np.log(-LOSS) )

print('totalTimeTrainingPDLVM = ', (endTrainingPDDLVM - startTrainingPDDLVM)/60 )


In [ ]:
# dirnameGT = 'data-groundTruth--PoissonNonLinear1D'
# dirnameGT = 'data-groundTruth--PoissonNonLinear1D_1000'
# dirnameGT = '../../fenics/exp/poisson1DNonLinear-Comparisons/data-groundTruth--PoissonNonLinear1D_1000'
dirnameGT = '../../../fenics/poisson1DNonLinear/data-groundTruth--PoissonNonLinear1D_1000'

groundTruthU = tf.constant( np.loadtxt(dirnameGT+'/' + 'allGroundTruthU.dat' ), dtype=tf.float32)
allTrueZs    = tf.constant( np.loadtxt(dirnameGT+'/' + 'allGroundTruthZ.dat' ), dtype=tf.float32)
allTrueWs    = tf.constant( np.loadtxt(dirnameGT+'/' + 'allGroundTruthW.dat' ), dtype=tf.float32)
groundTruthX = tf.constant( np.loadtxt(dirnameGT+'/' + 'FenicsMeshX.dat'     ), dtype=tf.float32)

saveDataList = True

if saveDataList:

     allresAbsMean = []

     allSampleZs   = []
     allMeanZs     = []
     allStdZs      = []

     allUSamples = []
     allUMeans   = []
     allUStd     = []

     allZNSE       = []
     allUNSE       = []

     Pin2SigK = []
     Pin2SigU = []

Xtest   = groundTruthX[:, None]
print('Xtest.shape = ', Xtest.shape)

tf_residualFunction = tf.function( residualFunction )
tf_alphaNet         = tf.function( alphaNet )
tf_betaNet          = tf.function( betaNet )

PLOT = False

for i in range(groundTruthU.shape[0]):
# for i in [2,3,4,7,8]:

     zTrue = allTrueZs[i]
     wTrue = allTrueWs[i]
     uTrue = groundTruthU[i]
     
     u_untrans, us, residual, logP_alpha_u_Z = tf_residualFunction( zTrue, wTrue, Xtest )
     mean_alpha, logvar_alpha = tf_alphaNet( zTrue, wTrue, Xtest )
     u_untrans = tf.reshape(reparameterize(mean_alpha, logvar_alpha), [-1,1])

     u = tf.squeeze( D( Xtest ) * u_untrans )
     mean_alpha_trans   = tf.squeeze( D( Xtest ) * mean_alpha )
     var_alpha_trans    = tf.squeeze( D( Xtest )**2. * tf.exp(logvar_alpha) )
     std_alpha_trans    = tf.sqrt( var_alpha_trans )
     # std_alpha_trans    = tf.squeeze( D( Xtest )**2. * tf.exp(0.5 * logvar_alpha) )

     if PLOT:
          c = next(plt.gca()._get_lines.prop_cycler)['color']

          plt.scatter(Xtest, u,                1,  color=c)#linewidth=1
          plt.plot(Xtest, mean_alpha_trans,  '--', linewidth=1,  c=c)
          plt.plot(Xtest, uTrue,   linewidth=1,    c=c)
          plt.fill_between(Xtest[:,0], mean_alpha_trans - 2*std_alpha_trans, mean_alpha_trans + 2*std_alpha_trans,color=c, alpha=0.2)
     
     
     mean_beta, logvar_beta = tf_betaNet( tf.reshape(uTrue/D(tf.squeeze(Xtest)), [-1,1]), wTrue,  Xtest )
     mean_beta, logvar_beta = tf.squeeze( mean_beta), tf.squeeze( logvar_beta )
     sampleZ  = reparameterize( mean_beta, logvar_beta )
     std_beta = tf.exp( 0.5 * logvar_beta)

     print('mean_beta = ', mean_beta)
     print('std_beta = ' , tf.exp( 0.5 * logvar_beta))
     print('true z = '   , zTrue)
     print('\n\n')


     if saveDataList:

          allresAbsMean.append( tf.reduce_mean( tf.abs( residual[1:-1] ) ) )

          allSampleZs.append(tf.squeeze(sampleZ))
          allMeanZs.append(tf.squeeze(mean_beta))
          allStdZs.append(tf.squeeze(tf.exp( 0.5 * logvar_beta)))

          allUSamples.append(tf.squeeze(us))
          allUMeans.append(tf.squeeze(mean_alpha_trans))
          allUStd.append(tf.squeeze(mean_alpha_trans))
          
          allZNSE.append( ( tf.linalg.norm((tf.squeeze(mean_beta) - tf.squeeze(zTrue)) ) / tf.linalg.norm(tf.squeeze(zTrue)) )**2. )

          allUNSE.append( ( tf.linalg.norm((tf.squeeze(mean_alpha_trans) - tf.squeeze(uTrue)) ) / tf.linalg.norm(tf.squeeze(uTrue)) )**2. )


          Pin2SigK.append( tf.reduce_sum(tf.where(tf.math.logical_and( 
               zTrue > mean_beta-2*std_beta, zTrue < mean_beta+2*std_beta
               ), 1, 0)) / zTrue.shape[0] )

          Pin2SigU.append( tf.reduce_sum(tf.where(tf.math.logical_and(
               uTrue > mean_alpha_trans-2*std_alpha_trans, uTrue < mean_alpha_trans+2*std_alpha_trans
               ), 1, 0)) / uTrue.shape[0]  )


# import matplotlib.lines as mlines
# from matplotlib.patches import Patch
# samples     = mlines.Line2D([], [],  color='blue', linestyle=':',  linewidth=1., label=r'$samples$')
# dashed      = mlines.Line2D([], [],  color='blue', linestyle='--', linewidth=1., label=r'$mean$')
# solid       = mlines.Line2D([], [],  color='blue', linestyle='-', linewidth=1.,    label=r'$truth$')
# fill        = mlines.Line2D([], [],  color='blue', linestyle='-', linewidth=5.,    label=r'$2\sigma$')
# fill        = Patch(facecolor='blue', edgecolor='blue', label='Color Patch')
# plt.legend(handles=[samples, dashed, solid, fill], loc='best') #best upper-right
if PLOT:
     plt.xlabel(r'$x$')
     plt.ylabel(r'$u(x)\pm2\sigma$')
     plt.tight_layout()
     # plt.savefig(dirname + 'samplePlotsTrusVdiaglfm.pdf')
     plt.show()

# allresAbsMeanStacked = tf.stack( allresAbsMean )

# print('mean allresAbsMeanStacked = ', tf.reduce_mean( allresAbsMeanStacked ) )

print('mean allZNSE = ', tf.reduce_mean( tf.stack(allZNSE) ) )
print('mean allUNSE = ', tf.reduce_mean( tf.stack(allUNSE) ) )

print('sttdev allZNSE = ', tf.math.reduce_std( tf.stack(allZNSE) ) )
print('sttdev allUNSE = ', tf.math.reduce_std( tf.stack(allUNSE) ) )

# print('median allZNSE = ', tfp.stats.percentile( allZNSE, 50 ) )

print('Pin2SigK = ', tf.reduce_mean(tf.stack(Pin2SigK)))
print('Pin2SigU = ', tf.reduce_mean(tf.stack(Pin2SigU)))
print('totalTimeTrainingPDLVM = ', (endTrainingPDDLVM - startTrainingPDDLVM)/60 )

raise

In [ ]:
if True:
    alphaNet.save_weights(dirname + '/alphaModel/alphaModel-lfmDiag')
    # alphaNet.load_weights(dirname + '/alphaModel/alphaModel-lfmDiag')

    # del alphaNet

if True:
    betaNet.save_weights(dirname + '/betaModel/betaModel-lfmDiag')
    # betaNet.load_weights(dirname + '/betaModel/betaModel-lfmDiag')

    # del betaNet


In [ ]:
saveData = True
if saveData:

    allSampleZs_np = tf.stack(allSampleZs)
    allMeanZs_np   = tf.stack(allMeanZs)
    allStdZs_np    = tf.stack(allStdZs)

    allUSamples_np  = tf.stack(allUSamples)
    allUMeans_np    = tf.stack(allUMeans)
    allUStd_np      = tf.stack(allUStd)

    prefix = 'randgrid{}_{}kIter'.format(nxSamples, int(num_iterations/1000))

    np.savetxt(dirname + prefix + '_allRes_np.dat'   , tf.stack(tf.squeeze(allresAbsMean)) )


    np.savetxt(dirname + prefix + '_allSampleZs_np.dat', allSampleZs_np)
    np.savetxt(dirname + prefix + '_allMeanZs_np.dat'  , allMeanZs_np)
    np.savetxt(dirname + prefix + '_allStdZs_np.dat'  , allMeanZs_np)


    np.savetxt(dirname + prefix + '_allUSamples_np.dat' , allUSamples_np)
    np.savetxt(dirname + prefix + '_allUMeans_np.dat'   , allUMeans_np)
    np.savetxt(dirname + prefix + '_allUStd_np.dat'     , allUStd_np)
    
    np.savetxt(dirname + prefix + '_allZNSE_np.dat'   , tf.stack( allZNSE ) )
    np.savetxt(dirname + prefix + '_allUNSE_np.dat'   , tf.stack( allUNSE ) )

In [ ]:
for layer in betaNet.NN.kipLayers:
    print(layer.ell)